# name of project/cruise/paper

## description

## year(s)

# Import libraries

In [11]:
import os
import numpy     as np
import pandas    as pd
import datetime

# For map
import map_setup           as map_setup
import matplotlib.pyplot   as plt
import cartopy.feature     as cfeature           # to add land features to map

# Load data

In [6]:
# chemtax = pd.read_excel('../../data/datasets/unmodified/KERFIX_1992-1995_France_JGOFS/KERFIX_cleaned.xlsx')

# xls               = pd.ExcelFile('../../data/datasets/unmodified/KERFIX_1992-1995_France_JGOFS/KERFIX_cleaned.xlsx')
# chl_and_taxa_raw  = pd.read_excel(xls, 'chl_and_taxa_2')
# T_S_nutrients_raw = pd.read_excel(xls, 'T_S_nutrients')

In [19]:
chl_and_taxa_raw.dtypes

Year                               float64
Month                               object
Day                                float64
Depth_m                            float64
Flagellates_x10^3/L                 object
Coccolithophores_x10^3/L            object
Dinoflagelates_x10^3/L              object
Diatoms_x10^3/L                     object
Total_x10^3/L                       object
Date                        datetime64[ns]
Z(m)                               float64
Chl a                              float64
Chl b                              float64
Chl c                              float64
Phe a                              float64
dégrad.                            float64
dtype: object

In [20]:
T_S_nutrients_raw.dtypes

Date                       datetime64[ns]
Temperature_degC                  float64
Temperature_degC_stdev             object
Salinity_psu                      float64
Salinity_psu_stdev                 object
MLD_m                               int64
nitrate_mM_perm3                  float64
nitrate_mM_perm3_stdev             object
silicate_mM_perm3                 float64
silicate_mM_perm3_stdev            object
dtype: object

# Organize data

In [23]:
# ######################## DATASETS ################################################################################
# ### Save raw data for manipulation
# chl_and_taxa          = chl_and_taxa_raw
# T_S_nutrients         = T_S_nutrients_raw
# hydrography_nutrients = hydrography_nutrients_raw
# biogenic_si           = biogenic_si_raw




# ######################## FORMAT NANs #############################################################################
# ### Convert n.d. (no data) to NaN
# chl_and_taxa  =  chl_and_taxa.replace('n.d.', np.nan).replace('n.d', np.nan)
# T_S_nutrients = T_S_nutrients.replace('n.d.', np.nan).replace('n.d', np.nan)




# ######################## RENAME COLUMNS ###########################################################################
# ### Rename columns
# chl_and_taxa_sophy = {'Date':'timestamp', 'depth_m': 'depth_taxa', 
#                       'Flagellates_x10^3/L':'flagellates', 'Coccolithophores_x10^3/L':'coccolithophyceae', 'Dinoflagelates_x10^3/L':'dinophyceae', 'Diatoms_x10^3/L':'bacillariophyceae', 'Total_x10^3/L':'total',
#                       'Z(m)':'depth_pigs', 'Chl a': 'chl_a_fluor', 'Chl b':'chl_b', 'Chl c':'chl_c', 'Phe a':'phaeophytin', 'dégrad.':'degraded_photopigments'}
# chl_and_taxa = chl_and_taxa.rename(columns=chl_and_taxa_sophy)


# hydrography_nutrients_sophy = {'PStd':'pressure_standard_levels', 'Pression':'pressure', 'PCorr':'pressure_corrected', 'Temp_in_situ':'temperature', 'Salinite':'salinity', 'Oxygene':'oxygen', 
#                      'TPOT':'potential_temperature', 'Sigma_theta':'sigma_theta', 'NO3':'nitrate', 'PO4':'phosphate', 'SiO2':'silicate', 'NH4':'ammonium', 
#                      'Chl a':'chl_a_fluor', 'TCO2':'tco2', 'PCO2':'pco2', 'Alcalinit?':'alkalinity'}
# hydrography_nutrients = hydrography_nutrients.rename(columns = hydrography_nutrients_sophy)


# biogenic_si_sophy = {'depth (m)':'depth', 'BSi (mmol/m^3)':'BSi'}
# biogenic_si = biogenic_si.rename(columns = biogenic_si_sophy)




# ######################## FORMAT DATATYPES ########################################################################
# ### Convert date
# # Date time already formatted in dataframes for chl_and_taxa and T_S_nutrients
# hydrography_nutrients['timestamp'] = hydrography_nutrients['Date'].apply(lambda x: pd.to_datetime(str(x), format='%Y%m%d'))

# biogenic_si['Month']     = pd.to_datetime(biogenic_si.Month, format='%B').dt.month   # Convert month name to integer )(i.e. March -> 3)
# biogenic_si['timestamp'] = pd.to_datetime(biogenic_si[['Year', 'Month', 'Day']])     # Combine Year, Month, and Day into one timestamp column


# ### Make sure datatypes are all float
# # All datatypes are correct




# ######################## DROP UNNEEDED COLUMNS ##################################################################
# ### Drop unneeded columns
# hydrography_nutrients = hydrography_nutrients.drop(columns=['Date', 'pressure', 'mes'])
# biogenic_si           = biogenic_si.drop(columns=['date', 'Year', 'Month', 'Day'])


# ### Drop rows without taxa
# chl_and_taxa = chl_and_taxa.dropna(subset=['total'])
# hydrography_nutrients = hydrography_nutrients.dropna(subset=['temperature', 'salinity', 'oxygen', 'chl_a_fluor', 'tco2', 'pco2', 'alkalinity'], how='all')




# ######################## UNITS ##################################################################################
# ### Make sure all units are the same
# # Taxa units are number *10^3 per liter
# chl_and_taxa['flagellates']       = chl_and_taxa['flagellates'].apply(lambda x: x * 10e3)
# chl_and_taxa['coccolithophyceae'] = chl_and_taxa['coccolithophyceae'].apply(lambda x: x * 10e3)
# chl_and_taxa['dinophyceae']       = chl_and_taxa['dinophyceae'].apply(lambda x: x * 10e3)
# chl_and_taxa['bacillariophyceae'] = chl_and_taxa['bacillariophyceae'].apply(lambda x: x * 10e3)

# # Make depths positive
# hydrography_nutrients['pressure_corrected'] = hydrography_nutrients['pressure_corrected'].apply(lambda x: x * -1)
# biogenic_si['depth'] = biogenic_si['depth'].apply(lambda x: x * -1)




# ######################## MERGE DATA FRAMES #####################################################################
# ### Merge dataframes together
# # Merge BSi into hydrography_nutrients
# biogenic_si.rename(columns={'depth':'pressure_standard_levels'}, inplace=True)
# hydrography_nutrients_bsi = hydrography_nutrients.merge(biogenic_si, on=['timestamp', 'pressure_standard_levels'], how='left')


# # Merge MLD from T_S_nutrients into hydrography_nutrients_bsi
# T_S_nutrients.drop(T_S_nutrients[['Temperature_degC', 'Temperature_degC_stdev', 'Salinity_psu', 'Salinity_psu_stdev', 'nitrate_mM_perm3', 'nitrate_mM_perm3_stdev', 'silicate_mM_perm3', 'silicate_mM_perm3_stdev']], axis=1, inplace=True)
# T_S_nutrients.rename(columns={'Date ':'timestamp'}, inplace=True)
# hydrography_nutrients_bsi_mld = hydrography_nutrients_bsi.merge(T_S_nutrients, on=['timestamp'], how='left')

# # Organize column order
# hydrography_nutrients_bsi_mld = hydrography_nutrients_bsi_mld[['timestamp', 'pressure_standard_levels', 'pressure_corrected', 'temperature', 'potential_temperature', 'salinity', 'sigma_theta', 
#                                                                'MLD_m', 'oxygen', 'nitrate', 'ammonium', 'phosphate', 'silicate', 'BSi', 'tco2', 'pco2', 'alkalinity']]


# # Merge in chl_and_taxa dataframe
# # Organize chl_and_taxa dataframe and drop unneeded columns
# chl_and_taxa = chl_and_taxa[['timestamp', 'Depth_m', 'chl_a_fluor', 'chl_b', 'chl_c', 'phaeophytin', 'degraded_photopigments', 'flagellates', 'coccolithophyceae', 'dinophyceae', 'bacillariophyceae', 'total']]
# chl_and_taxa.rename(columns={'Depth_m':'pressure_standard_levels'}, inplace=True)
# hydrography_nutrients_bsi_mld_taxa = hydrography_nutrients_bsi_mld.merge(chl_and_taxa, on=['timestamp', 'pressure_standard_levels'], how='left')

# # Drop rows without taxa
# hydrography_nutrients_bsi_mld_taxa = hydrography_nutrients_bsi_mld_taxa.dropna(subset=['total']).reset_index()




# ######################## SOPHY COLUMN NAMES ################################################
# ### Add in columns needed for SOPhy
# # Add source name
# hydrography_nutrients_bsi_mld_taxa.insert(0, 'source_name', 'KERFIX')

# # Add ID
# hydrography_nutrients_bsi_mld_taxa.insert(1, 'sample_id', hydrography_nutrients_bsi_mld_taxa.index + 1)

# # Add latitude and longitude
# hydrography_nutrients_bsi_mld_taxa.insert(2, 'latitude',  kerfix_latitude)
# hydrography_nutrients_bsi_mld_taxa.insert(3, 'longitude', kerfix_longitude)

# # Add in which data is available
# hydrography_nutrients_bsi_mld_taxa['hplc_present']       = 0
# hydrography_nutrients_bsi_mld_taxa['chemtax_present']    = 0
# hydrography_nutrients_bsi_mld_taxa['microscopy_present'] = 1
# hydrography_nutrients_bsi_mld_taxa['ifcb_present']       = 0

# # Set names to match SOPhy columns
# hydrography_nutrients_bsi_mld_taxa.rename(columns={'pressure_standard_levels':'depth'}, inplace=True)




# ######################## FORMAT FOR SCHEMA ################################################
# ### Make separate dataframe to match SOPhy schema
# kerfix_sample = hydrography_nutrients_bsi_mld_taxa[['source_name', 'sample_id', 'latitude', 'longitude', 'timestamp', 'depth', 'temperature', 'potential_temperature',
#                                                     'salinity', 'sigma_theta', 'MLD_m', 'oxygen', 'nitrate', 'ammonium', 'phosphate', 'silicate', 'BSi', 
#                                                     'tco2', 'pco2', 'alkalinity', 'chl_a_fluor', 'chl_b', 'chl_c', 'phaeophytin', 'degraded_photopigments',
#                                                     'hplc_present', 'chemtax_present', 'microscopy_present', 'ifcb_present']]

# kerfix_sample_amount_wide = hydrography_nutrients_bsi_mld_taxa[['source_name', 'sample_id', 'timestamp', 'depth', 'flagellates', 'coccolithophyceae', 'dinophyceae', 'bacillariophyceae']]


# # Format sample_amount dataframe
# kerfix_sample_amount = pd.melt(kerfix_sample_amount_wide, id_vars=['source_name', 'sample_id', 'timestamp', 'depth'], value_vars=['flagellates', 'coccolithophyceae', 'dinophyceae', 'bacillariophyceae'], var_name='taxa', value_name='cells_per_L')
# kerfix_sample_amount.sort_values(by=['sample_id', 'timestamp', 'depth'], inplace=True)
# kerfix_sample_amount = kerfix_sample_amount[['sample_id', 'taxa', 'cells_per_L']]

# kerfix_sample_amount['hplc_present']       = 0
# kerfix_sample_amount['chemtax_present']    = 0
# kerfix_sample_amount['microscopy_present'] = 1
# kerfix_sample_amount['ifcb_present']       = 0

# Make a map to show where data is

In [ ]:
############ DATA TO PLOT ################################################
dataset        = kerfix_sample
data_longitude = kerfix_longitude
data_latitude  = kerfix_latitude


############ SET UP PLOT ################################################

#### COLORS
data_point_color = '#FF4FBB'

#### FIGURE SIZE
fig_width  = 3   # inches
fig_height = 3   # inches
markersize = 5

#### CREATE FIGURE
fig  = plt.figure(figsize=[fig_width, fig_height], dpi=300) # inches
ax1  = plt.subplot(projection = map_setup.map_proj)



############ CREATE MAP ################################################

# Make map
map_setup.map_southern_ocean_axes_setup(ax1, fig, add_gridlines=True)

# Add fronts to map
map_setup.add_fronts(ax1, add_labels=True, front_linewidth=0.25, fontsize=4)

# Add land after front
ax1.add_feature(cfeature.LAND, zorder=7, linewidth = 1, edgecolor=map_setup.overplot_label_color, facecolor=map_setup.overplot_label_color)



############ ADD DATA TO MAP ################################################
plt.scatter(x=data_longitude, y=data_latitude, c=data_point_color, transform=map_setup.text_proj, zorder=6, s=markersize)



############ FORMAT TITLE ###################################################
# Get source name and number of observations
source_name_title = np.unique(dataset.source_name)[0]
n_obs = len(dataset.index)

# Find range of years
first_year = np.nanmin(dataset.timestamp).astype(str)[0:4]
last_year  = np.nanmax(dataset.timestamp).astype(str)[0:4]

if first_year != last_year:
    years = first_year + u'\u2013' + last_year
if first_year == last_year:
    years = first_year

# Find range of months
month_array = []
for i in range(0,len(dataset.index)):
    month = dataset.timestamp[i].month
    month_array.append(month)
        
first_month = np.nanmin(month_array)
last_month = np.nanmax(month_array)

months = calendar.month_abbr[first_month] + u'\u2013' + calendar.month_abbr[last_month]


plt.title(source_name_title, loc='left', fontsize=8)
plt.suptitle('Years: ' + years + '\nMonths: ' + months + '\nn obs = ' + str(n_obs), x=.05, y=.95, fontsize=5, ha='left')


plt.show()

In [ ]:
# How to look up strings in a data column:

hydrography_nutrients.chl_a_fluor[[isinstance(x, str) for x in hydrography_nutrients.chl_a_fluor]]